Question 1

In [182]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import folium
import numpy as np
from geopy.geocoders import Nominatim
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

In [120]:
res = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(res.content,'lxml')
table = soup.find_all('table')[0] 
df = pd.read_html(str(table))[0]
postcode = df["Postcode"].tolist()
borough = df["Borough"].tolist()
neighbourhood= df["Neighbourhood"].tolist()

In [121]:
#removing borough that are not assigned
df = df[df.Borough != 'Not assigned'] 

In [122]:
#Combing neighborhood with the same postcode
df2 = pd.DataFrame({'Postcode':df.Postcode.unique()})
df2['Borough']=pd.DataFrame(list(set(df['Borough'].loc[df['Postcode'] == x['Postcode']])) for i, x in df2.iterrows())
df2['Neighbourhood']=pd.Series(list(set(df['Neighbourhood'].loc[df['Postcode'] == x['Postcode']])) for i, x in df2.iterrows())
df2['Neighbourhood']=df2['Neighbourhood'].apply(lambda x: ', '.join(x))


# replace "Not assigned" Neighborhoods with Borough
df2.Neighbourhood.replace("Not assigned", df2.Borough, inplace=True)

In [124]:
df2.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Queen's Park


In [125]:
df2.shape

(103, 3)

Question 2

In [136]:
#add Geo-spatial data
dfll= pd.read_csv("http://cocl.us/Geospatial_data")
dfll.rename(columns={'Postal Code':'Postcode'}, inplace=True)
dfll.set_index("Postcode")
df2.set_index("Postcode")
toronto_data=pd.merge(df2, dfll)
toronto_data.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494


Question 3

In [138]:
address = 'Toronto'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [141]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [145]:
CLIENT_ID = 'S4CCVGHDBUD1GJCYMFSTBIEJRNGKAI3ABY0K2QQ15TCYONLG' # your Foursquare ID
CLIENT_SECRET = 'JBKXVLRA1IDD42I5N5IERQGFTJM3TICE2PNETZ3D2MYU1OYW' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

In [148]:
toronto_data.loc[0, 'Neighbourhood']
neighbourhood_latitude = toronto_data.loc[0, 'Latitude'] # neighborhood latitude value
neighbourhood_longitude = toronto_data.loc[0, 'Longitude'] # neighborhood longitude value

neighbourhood_name = toronto_data.loc[0, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighbourhood_name, 
                                                               neighbourhood_latitude, 
                                                               neighbourhood_longitude))

Latitude and longitude values of Parkwoods are 43.7532586, -79.3296565.


In [156]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [157]:
toronto_venues = getNearbyVenues(names=toronto_data['Neighbourhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )


Parkwoods
Victoria Village
Harbourfront
Lawrence Heights, Lawrence Manor
Queen's Park
Queen's Park
Rouge, Malvern
Don Mills North
Parkview Hill, Woodbine Gardens
Ryerson, Garden District
Glencairn
West Deane Park, Islington, Cloverdale, Martin Grove, Princess Gardens
Highland Creek, Port Union, Rouge Hill
Don Mills South, Flemingdon Park
Woodbine Heights
St. James Town
Humewood-Cedarvale
Bloordale Gardens, Eringate, Markland Wood, Old Burnhamthorpe
West Hill, Guildwood, Morningside
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Thorncliffe Park
Adelaide, Richmond, King
Dufferin, Dovercourt Village
Scarborough Village
Henry Farm, Fairview, Oriole
Northwood Park, York University
East Toronto
Harbourfront East, Toronto Islands, Union Station
Trinity, Little Portugal
Ionview, East Birchmount Park, Kennedy Park
Bayview Village
CFB Toronto, Downsview East
The Danforth West, Riv

In [159]:
toronto_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
"Adelaide, Richmond, King",100,100,100,100,100,100
Agincourt,4,4,4,4,4,4
"Agincourt North, L'Amoreaux East, Steeles East, Milliken",3,3,3,3,3,3
"Bathurst Manor, Wilson Heights, Downsview North",20,20,20,20,20,20
Bayview Village,4,4,4,4,4,4
Berczy Park,57,57,57,57,57,57
"Birch Cliff, Cliffside West",4,4,4,4,4,4
"Bloordale Gardens, Eringate, Markland Wood, Old Burnhamthorpe",10,10,10,10,10,10
Business Reply Mail Processing Centre 969 Eastern,19,19,19,19,19,19


In [160]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 277 uniques categories.


In [161]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighbourhood'] = toronto_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighbourhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [162]:
toronto_onehot.shape

(2241, 278)

In [163]:
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_grouped

,Neighbourhood,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,"Adelaide, Richmond, King",0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.020000,...,0.00,0.020000,0.0,0.00,0.000000,0.000000,0.010000,0.000000,0.0,0.000000
1,Agincourt,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.000000,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
2,"Agincourt North, L'Amoreaux East, Steeles East...",0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.000000,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
3,"Bathurst Manor, Wilson Heights, Downsview North",0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.000000,0.0,0.05,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
4,Bayview Village,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.000000,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
5,Berczy Park,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.017544,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
6,"Birch Cliff, Cliffside West",0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.000000,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
7,"Bloordale Gardens, Eringate, Markland Wood, Ol...",0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.000000,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.0,0.000000
8,Business Reply Mail Processing Centre 969 Eastern,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.000000,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.0,0.052632
9,"CFB Toronto, Downsview East",0.0,0.000000,0.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.000000,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.0,0.000000


In [164]:
toronto_grouped.shape

(100, 278)

In [168]:
num_top_venues = 5

for hood in toronto_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, Richmond, King----
          venue  freq
0   Coffee Shop  0.08
1          Café  0.05
2           Bar  0.04
3    Steakhouse  0.04
4  Burger Joint  0.03


----Agincourt----
                       venue  freq
0               Skating Rink  0.25
1                     Lounge  0.25
2             Breakfast Spot  0.25
3  Latin American Restaurant  0.25
4              Metro Station  0.00


----Agincourt North, L'Amoreaux East, Steeles East, Milliken----
               venue  freq
0         Playground  0.33
1               Park  0.33
2        Coffee Shop  0.33
3  Accessories Store  0.00
4      Metro Station  0.00


----Bathurst Manor, Wilson Heights, Downsview North----
         venue  freq
0  Coffee Shop  0.10
1         Bank  0.05
2        Diner  0.05
3  Gas Station  0.05
4  Supermarket  0.05


----Bayview Village----
                 venue  freq
0                 Bank  0.25
1  Japanese Restaurant  0.25
2   Chinese Restaurant  0.25
3                 Café  0.25
4    Accessories Stor

4                   Dog Run   0.2


----Humber Summit----
                        venue  freq
0         Empanada Restaurant   0.5
1               Shopping Mall   0.5
2           Accessories Store   0.0
3  Modern European Restaurant   0.0
4           Mobile Phone Shop   0.0


----Humbergate, Jamestown, Silverstone, Mount Olive, Thistletown, Beaumond Heights, South Steeles, Albion Gardens----
                 venue  freq
0        Grocery Store  0.25
1          Pizza Place  0.12
2  Fried Chicken Joint  0.12
3             Pharmacy  0.12
4       Sandwich Place  0.12


----Humewood-Cedarvale----
                             venue  freq
0                            Trail  0.33
1                            Field  0.33
2                     Hockey Arena  0.33
3                Accessories Store  0.00
4  Molecular Gastronomy Restaurant  0.00


----Ionview, East Birchmount Park, Kennedy Park----
                venue  freq
0      Discount Store  0.29
1   Convenience Store  0.14
2  Chinese Restaura

4   Summer Camp   0.2


----Summerhill West, Forest Hill SE, South Hill, Rathnelly, Deer Park----
                venue  freq
0                 Pub  0.13
1         Coffee Shop  0.13
2        Liquor Store  0.07
3  Light Rail Station  0.07
4          Sports Bar  0.07


----The Beaches----
               venue  freq
0                Pub   0.2
1              Trail   0.2
2  Health Food Store   0.2
3       Neighborhood   0.2
4        Coffee Shop   0.2


----The Beaches West, India Bazaar----
            venue  freq
0            Park  0.10
1  Sandwich Place  0.10
2       Pet Store  0.05
3    Intersection  0.05
4   Burrito Place  0.05


----The Danforth West, Riverdale----
                    venue  freq
0        Greek Restaurant  0.20
1             Coffee Shop  0.10
2          Ice Cream Shop  0.07
3      Italian Restaurant  0.05
4  Furniture / Home Store  0.05


----The Junction North, Runnymede----
                             venue  freq
0                      Pizza Place  0.25
1           

In [195]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighbourhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, Richmond, King",Coffee Shop,Café,Bar,Steakhouse,Sushi Restaurant,Thai Restaurant,Salad Place,Restaurant,Burger Joint,Bakery
1,Agincourt,Lounge,Breakfast Spot,Latin American Restaurant,Skating Rink,Electronics Store,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant
2,"Agincourt North, L'Amoreaux East, Steeles East...",Park,Playground,Coffee Shop,Eastern European Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant
3,"Bathurst Manor, Wilson Heights, Downsview North",Coffee Shop,Deli / Bodega,Chinese Restaurant,Bridal Shop,Fast Food Restaurant,Sandwich Place,Diner,Restaurant,Supermarket,Bank
4,Bayview Village,Café,Bank,Chinese Restaurant,Japanese Restaurant,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Yoga Studio


In [196]:
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 4, 1, 1, 1, 1, 1, 1, 4])

In [204]:
# add clustering labels
#neighbourhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
toronto_merged = toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighbourhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

toronto_merged=toronto_merged.dropna() # check the last columns!
toronto_merged

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,4.0,Park,Food & Drink Shop,Yoga Studio,Electronics Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant
1,M4A,North York,Victoria Village,43.725882,-79.315572,1.0,Portuguese Restaurant,Intersection,Pizza Place,Coffee Shop,Hockey Arena,Dumpling Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636,1.0,Coffee Shop,Park,Pub,Bakery,Mexican Restaurant,Café,Performing Arts Venue,Brewery,French Restaurant,Ice Cream Shop
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763,1.0,Clothing Store,Furniture / Home Store,Accessories Store,Coffee Shop,Miscellaneous Shop,Athletics & Sports,Gift Shop,Boutique,Vietnamese Restaurant,Women's Store
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494,1.0,Coffee Shop,Gym,Diner,Sushi Restaurant,Park,Chinese Restaurant,Smoothie Shop,Seafood Restaurant,Sandwich Place,Burger Joint
5,M9A,Downtown Toronto,Queen's Park,43.667856,-79.532242,1.0,Coffee Shop,Gym,Diner,Sushi Restaurant,Park,Chinese Restaurant,Smoothie Shop,Seafood Restaurant,Sandwich Place,Burger Joint
6,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353,1.0,Fast Food Restaurant,Yoga Studio,Eastern European Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Electronics Store
7,M3B,North York,Don Mills North,43.745906,-79.352188,1.0,Café,Caribbean Restaurant,Gym / Fitness Center,Baseball Field,Japanese Restaurant,Basketball Court,Eastern European Restaurant,Donut Shop,Drugstore,Dumpling Restaurant
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937,1.0,Fast Food Restaurant,Pizza Place,Pet Store,Athletics & Sports,Gastropub,Intersection,Pharmacy,Breakfast Spot,Bank,Gym / Fitness Center
9,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937,1.0,Coffee Shop,Clothing Store,Café,Middle Eastern Restaurant,Cosmetics Shop,Theater,Diner,Plaza,Bubble Tea Shop,Italian Restaurant


In [205]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Cluster 1

In [206]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
62,Central Toronto,0.0,Garden,Yoga Studio,Eastern European Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Electronics Store


cluster 2

In [207]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,North York,1.0,Portuguese Restaurant,Intersection,Pizza Place,Coffee Shop,Hockey Arena,Dumpling Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop
2,Downtown Toronto,1.0,Coffee Shop,Park,Pub,Bakery,Mexican Restaurant,Café,Performing Arts Venue,Brewery,French Restaurant,Ice Cream Shop
3,North York,1.0,Clothing Store,Furniture / Home Store,Accessories Store,Coffee Shop,Miscellaneous Shop,Athletics & Sports,Gift Shop,Boutique,Vietnamese Restaurant,Women's Store
4,Queen's Park,1.0,Coffee Shop,Gym,Diner,Sushi Restaurant,Park,Chinese Restaurant,Smoothie Shop,Seafood Restaurant,Sandwich Place,Burger Joint
5,Downtown Toronto,1.0,Coffee Shop,Gym,Diner,Sushi Restaurant,Park,Chinese Restaurant,Smoothie Shop,Seafood Restaurant,Sandwich Place,Burger Joint
6,Scarborough,1.0,Fast Food Restaurant,Yoga Studio,Eastern European Restaurant,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Electronics Store
7,North York,1.0,Café,Caribbean Restaurant,Gym / Fitness Center,Baseball Field,Japanese Restaurant,Basketball Court,Eastern European Restaurant,Donut Shop,Drugstore,Dumpling Restaurant
8,East York,1.0,Fast Food Restaurant,Pizza Place,Pet Store,Athletics & Sports,Gastropub,Intersection,Pharmacy,Breakfast Spot,Bank,Gym / Fitness Center
9,Downtown Toronto,1.0,Coffee Shop,Clothing Store,Café,Middle Eastern Restaurant,Cosmetics Shop,Theater,Diner,Plaza,Bubble Tea Shop,Italian Restaurant
12,Scarborough,1.0,History Museum,Bar,Yoga Studio,Electronics Store,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Empanada Restaurant
